MIT License

Copyright (c) 2021 Rafał Pracht

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

## Zadanie konkursowe

W ramach finałowego zadania należy znaleźć możliwe lokalizacje nadajników dla poniższej mapy stacji meteorologicznych. Pamiętaj, że stacja może przesyłać dane najdalej do nadajnika przy stacji, z którą sąsiaduje.

<img src="graphics/task4.png" width=400/>

Skądinąd wiemy, że wystarczy ustawić dokładnie trzy nadajniki, co więcej, minimum trzy nadajniki są potrzebne do rozwiązania problemu.

**Rozwiązanie musi spełniać szereg reguł**:

* Skonstruowany obwód nie może składać się z więcej niż 26 kubitów.
* Należy użyć algorytmu Grovera, który poznaliście w zadaniu drugim, z trzema iteracjami.
* Wynik pomiaru $1$ oznacza, że wierzchołek należy do zbioru dominującego, a wynik $0$, że nie należy.
* Zastosuj jedynie jeden 9-bitowy klasyczny rejestr `c`, w którym są zapisywane wyniki pomiaru. W tym celu możecie korzystać z wzorca poniżej.
* __Uwaga: można skorzystać z informacji, że szukamy położenia trzech nadajników, jednak jest to jedyna informacja dotycząca rozwiązania, z której można skorzystać__. W szczególności nie można implementować wyroczni poprzez wcześniejsze znalezienie rozwiązania klasycznymi metodami. **Pomocnicze pytania:** Czy algorytm który skonstruowałem/am będzie działał dla dowolnego grafu nieskierowanego, jeśli będę znał/a rozmiar zbioru dominującego (powinno być: **tak**)? Czy korzystam z jakichś nietypowych cech grafu (jest nieregularny, niedwudzielny itp.) (powinno być: **nie**)? Czy korzystam z wiedzy jakie jest rozwiązanie (powinno być: **nie**)? 
* Obwód kwantowy może składać się jedynie z dostępnych w Qiskit bramek kwantowych (`x`, `ccx`, `mct`, `z`, `u3`, etc.) oraz pomiarów. Nie można korzystać z operacji nie będących bramkami, typu `reset`, `c_if`, itp, oraz bramki `unitary`. Jeśli masz wątpliwości co do użytej bramki - zapytaj mentorów.
* Nie można korzystać z optymalizatorów do obwodów kwantowych opartych na uczeniu maszynowych bądź heurystycznych innych niż `transpile` użyty poniżej. W `transpile` poniżej nie można zmieniać argumentów.
* **Jako rozwiązanie należy wysłać plik `zadanie_4.json` wygenerowany przez `create_submission` oraz plik `zadanie_4.ipynb` wraz z odpowiednimi komentarzami opisującymi kroki algorytmu.**


In [1]:
from qiskit import *
from qiskit.providers.aer import QasmSimulator
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller
from bnp_challenge_verifier import grade_circuit, verify_solution4, create_submission
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_) 

## Opis rozwiązanie zadania.

**Ogólna koncepcja**:

Implementacja wyroczni składa się z dwóch sumatorów:
* Jeden zlicza liczbe oznaczonych wierzchołków.
* Drugi zlicza liczbe porawnych wierzchołków. Tzn. wierzchołków oznaczonych lub mających za sąsiada wierzchołek oznaczony.
Wiemy że liczba oznaczonych wierzchołków wynosi 3 a liczba poprawnych ma wynieść 9, wiedza ta pozwala nam na sprawdzenie czy dane rozwiązanie jest poprawne.

**Ogólna opytmalizacja**:

Wszędzie gdzie się dało bramki ccx, mcx zostały zamienione na ich odpowiedniki rccx które działają tak samo z dokładnością do globalnej fazy (wszędzie tam gdzie operacje ccx będą odwracane można spokojnie skorzystać z rcx).

Pozwoliło to na zbudowanie ukłądu o koszcie około 29 000.

**Opytmalizacja pod problem**:

Zastosowana poniżej opisana optymalizacja pozwoliła na uzyskanie finalnego wyniku, ale sprawia że kod zadziała tylko dla grafu o 3 rozwiązaniach i 9 wierzchołków (co jest zgodnie z treścią zadania).

* Dla sumowania do 9 potrzebny jest sumator 4 bitowy. Jednakże jeżeli szukamy sumy równej 0011 i jednocześnie wiemy że nie ma wartości powyżej 1001 to możemy ograniczyć się jedynie do 3 bitów, i sumować modulo 8. Pozwala to na wykorzystanie sumatora 3 bitowego zamiast 4 bitowego o mniejszym koście oraz z wykorzystaniem 1 qubitu mniej co pozwoli z koleji na nie odwracanie "jednej" dodatkowej operacji.
* Drugi sumator też może być 3 bitowy, wynika to z faktu iż 9%8 to 001 a pierwszy sumator zapewnia nam ze jest co najmniej 3 oznaczone wierzchołki, co oznacze ze jak suma wynosi 001 to jest 9%8 a nie po prostu 1. 
* Ponadto w sumatorach możemy zignorować dla ostatniego 9 bitu całą operacjie z przeniesieniem i skorzystać tylko z bramki cx.

**Opytmalizacja sumatorów**:

Ponieważ celem wykorzystywanych sumatorów jest zliczanie a nie sumowanie dwóch licz binarnych, możemy skorzystać z kwantowych odpowiedników dzielników częstotliwości a nie kalsycznych sumatorów, pozwala nam to na dodatkowe ograniczenie kosztu (poprzez brak rejestrów carry oraz mniejszy koszt sumatora).

**Graf**

Poniżej z krawędzi grafu buduję mape wierzchołków.

In [2]:
# krawędzie w grafie
edges = [[0, 1], [1, 7], [7, 6], [6, 0], [2, 3], [3, 8], [8, 7], [7, 2], [4, 5], [5, 6], [6, 8], [8, 4]]

In [3]:
# budujemy mape wierzchołkow
connections = {}
for e1, e2 in edges:
    s = connections.setdefault(e1, set())
    s.add(e2)
    s = connections.setdefault(e2, set())
    s.add(e1)
# dodajemy pusty set jak wierzchołek nie ma krawędzi
for i in range(9):
    connections.setdefault(i, set())

**Flaga**

Jej znaczenie zostanie opisane dlalej. Ustawienie jej na true da nam rozwiązanie akceptowalne dla gradera o koszcie 14445 i działające dla **Większości** grafuów. Ponadto problem dotyczy tylko wysp o dokładnie 3 wierzchołkach, które można bardzo prosto znaleść raz przechodząc mape connections i w zależności od grafu wykorzystać jedną bądź drugą scieżke. Co oznacza że punkt "Czy algorytm który skonstruowałem/am będzie działał dla dowolnego grafu nieskierowanego" był by spełniony. Nie jestem jednak pewien czy rozwiązanie takie było by zakceptowane, dlatego nie korzystam z tej optymalizacji.

**W rozwiązaniu przesłanym do oceny jest ona ustawiona na flase**

In [4]:
#wrong_solution = True
wrong_solution = False

In [5]:
qgrover = QuantumRegister(9, 'nodes')
qReg = qgrover
cgrover = ClassicalRegister(9)
####
qAnc = QuantumRegister(17, 'ancilla')  # 17

# Count and Dominating
if wrong_solution:
    qSum = qAnc[:3]
    qDominatingSum = qAnc[3:6]
    qNodeCheck = qAnc[6:15]
    qDominatingCountAnc = qAnc[16:]
    qPhaseAnc = qAnc[15:]
else:
    qSum = qAnc[:3]
    qDominatingSum = qAnc[3:6]
    qNodeCheck = qAnc[6:14]
    qDominatingCountAnc = qAnc[15:]
    qPhaseAnc = qAnc[14:]

####
qcirc = QuantumCircuit(qgrover, cgrover, qAnc)

**Sumator 3 bitowy - zliczający liczbę oznaczonych/poprawnych wierzchołków w zadaniu**

In [6]:
def sum3bits(qc, i, toAdd, qSum):
    if i == 0:
        qc.cx(toAdd, qSum[0])
    elif i < 3:
        qc.rccx(toAdd, qSum[0], qSum[1])
        qc.cx(toAdd, qSum[0])
    elif i < 8:
        qc.rcccx(toAdd, qSum[0], qSum[1], qSum[2])
        qc.rccx(toAdd, qSum[0], qSum[1])
        qc.cx(toAdd, qSum[0])
    else:
        # mamy tylko 9 wierzcholkow wiec tu wystarczy tylko flip
        # tzn, suma oznaczonych wierzcholkow moze byc bledna, ale na pewno nie bedzie wynosila 3 jak
        # nie jest to prawda
        qc.cx(toAdd, qSum[0])

**Wykorzystanie sumatora do sprawdzenia czy mamy tylko 3 oznaczone wierzchołki**

In [7]:
def create_count_circuit(qReg, qSum):
    qc = QuantumCircuit(qReg, qSum)

    for i in range(len(qReg)):
        sum3bits(qc, i, qReg[i], qSum)
    return qc

**Nowa bramki**

Bramki rccccx i rcccccx są analogiczne jak rcccx ale obsługuje większą liczbę qbitów kontrolnych.

In [8]:
def rcccccx(self, c0, c1, c2, c3, c4, t, a0):
    self.rcccx(c0, c1, c2, a0)
    self.rcccx(a0, c3, c4, t)
    self.rcccx(c0, c1, c2, a0)
    
QuantumCircuit.rcccccx = rcccccx

In [9]:
def rccccx(self, c0, c1, c2, c3, t, a0):
    self.rccx(c0, c1, a0)
    self.rcccx(a0, c2, c3, t)
    self.rccx(c0, c1, a0)
    
QuantumCircuit.rccccx = rccccx

**Sumator zliczający liczbę poprawnych wierzchołków.**

Algorytm dziła nasępująco:
* W pierwszym etapie na podstawie listy wierzchołków przygotowana jest mapa, mapująca index wierzchołka na zbór jego sąsiadów.
* Nasępnie, przechodzimy przeż wszystkie wierzchołki.
    * Sprawdzamy czy wierzchołek jest poprawny (czyli czy wszyscy jego sąsiedzi i on sam nie są ustawioni na 0)
    * Sprawdzenie może być ogólne poprzez bramkę mcx, ale w naszym grafie zawsze dokonujemy tego o przypadki szczególne rcccx i rcccccx. Zastosowanie rcccx zamiast mcx nie ma wpływu na algorymt a jedynie na jego koszt.
    * Dodajemy uzyskane sprawdzenie do sumatora (rozwiązanie poprawne ma wartość sumy równą 9%8, czyli wszystkie wierzchołki)
    * Ponieważ wiemy że nie ma więcej niż 9 wierzchołków, dla ostatniego bitu wykorzystujemy bramke cx zamiast sumatora z przeniesieniem.
    * Tablica qNodeCheck przechowuje wierzchołki które mogą pozostać _brudne_ po zakończeniu obliczeń. Dzięki temu oszczędzamy na odwracaniu niektórych operacji.
    * Przed przystąpień do sprawdzenia, szukamy czy w grafie nie wystepuja 3 sąsiednie wierzchołki. Ponieważ wierzchołek jest poprawny jak nie jest parwdą iż on i wszyscy jego sasiedzi są zerami więc to sprawdzenie jest wykonywane dla wszystkich sonsiednich wierzchołków. Jezeli mamy zbór 3 wierzchołków które są są siadami to możemy sprawdzenie dla nich wykonać tylko raz, a póżniej robić tylko sprawdzenia dla pozostałych wierzchołków. **UWAGA** zastosowanie sprawdzenia nie ma wpływu na znalezienie rozwiązania, tzn graf może nie mieć takich sonsiadów i poprawnie rozwiązanie zostanie znależione, jedynym skutkiem bedzie nie skorzystanie z możliwości optymalizacji. Można wyłaczyć optymalizacje ustawiając flage disable_cycle_opt = True daje to koszzt 16638 (przy rozwiazaniu ogólnym).

In [10]:
disable_cycle_opt = False
#disable_cycle_opt = True

In [11]:
def create_dominating_circuit_sum(edges, qReg, qSum, qNodeCheck, qAnc):
    qc = QuantumCircuit(qReg, qSum, qNodeCheck, qAnc)
        
    # znajdujem cykle 3 elemetow, dzieki temo bedziemy mogli policzyc je raz a nie dla kazdego wierzcholka osobno
    cycles = set()
    for i in range(len(qReg)):
        for j in range(len(qReg)):
            if i != j:
                s1 = set(list(connections[i]) + [i])
                s2 = set(list(connections[j]) + [j])
                inter = s1.intersection(s2)
                if len(inter) == 3:
                    # tylko dla 3 mozemy cokolwiek ugrac
                    cycles.add(frozenset(inter))        
        
    qc.x(qReg)
    # to ma wplyw tylko na mniejszy koszt, powinnismy isc to petelka bo cycles moze mniec wiecej niz jeden element
    # ale to by skaplikowalo kod ponizej a w naszym problemie jest dokladnie jeden, wiec ograniczamy potecjalny
    # zysk z optymalizacji dla innych grafów gdzie tych cykli jest wiecej. Jednak nie ma to wplywu na to ze sam
    # algorytm zadziala.
    ancIdx = 0
    cycle = []
#     for c in cycles:
#         print(c)
#         qc.rcccx(qReg[c[0]], qReg[c[1]], qReg[c[2]], qAnc[ancIdx])
#         ancIdx = ancIdx + 1
    if not disable_cycle_opt and len(cycles) > 0:
        cycle = list(list(cycles)[0])
        qc.rcccx(qReg[cycle[0]], qReg[cycle[1]], qReg[cycle[2]], qAnc[0])
        ancIdx = 1
    
    for i in range(len(qReg)):
        # chieck if node is ok
        control = set(list(connections[i]) + [i])
        diff = control.difference(cycle)
        if not disable_cycle_opt and len(diff) + len(cycle) == len(control):
            control = [qReg[c] for c in diff] + [qAnc[0]]
        else:
            control = [qReg[c] for c in control]
       
        idx = i -(len(qReg) - len(qNodeCheck))
        if idx >= 0:
            iNode = idx
        else:
            iNode = 0
        
        if len(control) == 3:
            qc.rcccx(control[0], control[1], control[2], qNodeCheck[iNode])
        elif len(control) == 5:
            qc.rcccccx(control[0], control[1], control[2], control[3], control[4], 
                      qNodeCheck[iNode],
                      qAnc[ancIdx])
        else:
            print("WARNING: the not optimal solution!")
            qc.mcx(control, qNodeCheck[iNode], qAnc, mode='basic')
        qc.x(qNodeCheck[iNode])

        # dodajemy
        sum3bits(qc, i, qNodeCheck[iNode], qSum)

        # reverse
        if idx < 0:
            qc.x(qNodeCheck[iNode])
            if len(control) == 3:
                qc.rcccx(control[0], control[1], control[2], qNodeCheck[iNode])
            elif len(control) == 5:
                qc.rcccccx(control[0], control[1], control[2], control[3], control[4], 
                          qNodeCheck[iNode],
                          qAnc[ancIdx])
            else:
                print("WARNING: the not optimal solution!")
                qc.mcx(control, qNodeCheck[iNode], qAnc, mode='basic')        
    if not disable_cycle_opt and len(cycles) > 0:        
        qc.rcccx(qReg[cycle[0]], qReg[cycle[1]], qReg[cycle[2]], qAnc[0])
    qc.x(qReg)

    return qc

**Wyrocznia**

Oracle to prasta kompozycja wcześniejszych sumatorów.

In [12]:
count_circuit = create_count_circuit(qReg, qSum)
dominating_circuit = create_dominating_circuit_sum(edges, qReg, qDominatingSum, qNodeCheck, qDominatingCountAnc)

oracle_circuit = QuantumCircuit(qReg, qAnc)

oracle_circuit += count_circuit
oracle_circuit += dominating_circuit

<ipython-input-12-0e8079c8f4e5>:6: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  oracle_circuit += count_circuit
/home/rpracht/anaconda3v02/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:876: DeprecationWarning: The QuantumCircuit.extend() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  return self.extend(rhs)


**Operator dyfuzji**

In [13]:
#operator dyfuzji
def inversion_about_average(circuit, register, ancilla):
    """Stosuje operację odbicia względem średniej algorytmu Grovera (Operacja 1)."""
    circuit.h(register)
    circuit.x(register)
    circuit.h(register[-1])
    circuit.mct(register[:-1], register[-1], ancilla, mode='basic')
    circuit.h(register[-1])
    circuit.x(register)
    circuit.h(register)

**Algorymt Grover'a**

Jest on standardowy. Sprawdzamy w nim czy mamy 3 oznaczone wierzchołki (011) oraz czy wszystkie wierzchołki są poprawne (001).

**UWAGA** Bardzo kuszące jest sprawdzanie tylko qbitów 0 i 2 z wyniku sumatora zliczającego poprawne wierzchołki. Wynika to z faktu iż szukana przez nas suma to 9 czyli b1001. Ponieważ sumowanie wykonywane jest modulo to oczekiwana wartość wynosi b001. Kożystając jednocześnie z faktu iż pierwszy sumator zapewnia nam że co najmniej 3 wierzchołki są oznaczone, wówacz mogą wystąpić tylko takie kombinacje:
* 000 - 8 dobrych wierzchołków (0 dobrych wierzchołków nie wystąpi gdyż muszą być co najmiej 3)
* 001 - 9 dobrych wierzchołków (1 dobrych wierzchołków nie wystąpi gdyż muszą być co najmiej 3)
* 010 - taka suma nie wystąpi dla grafu z 9 wierzchołkami i 3 oznaczonymi
* 011 - 3 dobre wierzchołki - sytuacja może wystąpić tylko dla grafu z wyspą 3 wierzchołków połączonych ze sobą i nie połączonych z innymi. Dla większości grafów (w tym dla naszego) algorytm zadziała poprawnie osiągając koszt 14445.
* 1xx - pozostale casy

Wówczas ignorując case 011 który może wystąpić tylko w przypadku wyspy można sprawdzać tylko czy drugi sumator zwrócił 0x1.

In [14]:
qcirc.h(qgrover)

for _ in range(3):
    qcirc += oracle_circuit

    # phase kick back
    # sprawdź, czy suma wynosi 3 => mamy tylko trzy nadajniki
    # sprawdź, czy suma wynosi 9 => wszystkie wierzcholki są ok
    qcirc.x(qSum[2])
    if wrong_solution:
        sumAll = qSum[:3] + [qDominatingSum[2], qDominatingSum[0]]
        qcirc.x(qDominatingSum[2])
    else:
        sumAll = qSum[:3] + qDominatingSum[:3]
        qcirc.x(qDominatingSum[1])
        qcirc.x(qDominatingSum[2])
    
    qcirc.h(sumAll[-1])
    qcirc.mcx(sumAll[:-1], sumAll[-1], qPhaseAnc, mode='basic')
    qcirc.h(sumAll[-1])
    
    if wrong_solution:
        qcirc.x(qDominatingSum[2])
    else:
        qcirc.x(qDominatingSum[2])
        qcirc.x(qDominatingSum[1])
    qcirc.x(qSum[2])
    

    # inv
    qcirc += oracle_circuit.inverse()

    # diffuse
    inversion_about_average(qcirc, qgrover, qAnc[:6])

In [15]:
qcirc.measure(qgrover, cgrover)

# weryfikacja rozwiązania
cost = grade_circuit(qcirc)

qcirc = transpile(qcirc, basis_gates=['u3', 'cx']) 
qcirc = pm.run(qcirc)
qasm = QasmSimulator(seed_simulator=46) # nie zmieniaj wartości seed_simulator!
result = qasm.run(qcirc).result().get_counts(qcirc)
# print(result)
verify_solution4(result)

Koszt twojego obwodu wynosi 15042.
Gratulacje, twój obwód wygenerował poprawne wyniki pomiaru! Upewnij się, że twój obwód spełnia pozostałe wymogi zadania.


Jeśli Twoje rozwiązanie zostało zaakceptowane przez nasz weryfikator, stwórz i prześlij plik zgłoszeniowy **razem z tym notebookiem uzupełnionym o Twoje rozwiązanie wraz z komentarzami** (łącznie dwa pliki). Z Twoich komentarzy powinno być jasne jak działa Twoje rozwiązanie. Upewnij się, że w `twoje_id` poniżej podałeś/aś swój poprawny identyfikator!

Zgłoszeń można dokonać [tutaj](https://ibm.ent.box.com/f/4c9101c0616f4897920a02a67d077321).

In [16]:
twoje_id = "8dba80cdf2"
create_submission(qcirc, result, twoje_id)

Wygenerowaliśmy plik do wysłania o nazwie: zgloszenie_konkursowe.json. Nie zapomnij wysłać notebooka z rozwiązaniem!
